# 第5章: 大規模言語モデル

この章では、大規模言語モデル (LLM; Large Language Model) の利用し、様々なタスクに取り組む。大規模言語モデルをプログラムからAPI経由で呼び出すことを想定しており、そのAPIの利用で費用が発生する可能性があることに留意せよ。

## 40. Zero-Shot推論

以下の問題の解答を作成せよ。ただし、解答生成はzero-shot推論とせよ。

```
9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB 問題](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 日本史B 1 問3

In [11]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
api_key = os.environ["GOOGLE_API_KEY"]
genai.configure(api_key=api_key)

model = genai.GenerativeModel(model_name="gemini-1.5-flash")
query = """9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

        ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
        イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
        ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。"""

zero_shot_prompt = f"質問: {query}\n回答: "
zero_shot_response = model.generate_content(zero_shot_prompt)

print("Zero-shot推論の結果:")
print(zero_shot_response.text)

Zero-shot推論の結果:
正しい並びは **イ→ウ→ア** です。


* **イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。**  嵯峨天皇の治世は809年～823年です。藤原冬嗣は806年に蔵人頭になっています。

* **ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。** 承和の変は842年。その後の藤原良房の勢力拡大によって北家が優勢になります。

* **ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。** これは、延喜の変（894年）のことです。


よって、年代順にイ、ウ、アとなります。



## 41. Few-Shot推論

以下の問題と解答を与え、問題40で示した質問の解答をfew-shot推論（この場合は4-shot推論）で生成せよ。

```
日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。

解答: ウ→イ→ア
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB 問題](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 日本史A 1 問8


```
江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。

ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。
イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。
ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。

解答: ウ→ア→イ
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB 問題](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 日本史B 3 問3

```
中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　アヘン戦争がおこり，清がイギリスに敗北した。
イ　異国船打払令が出され，外国船を撃退することが命じられた。
ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。

解答: イ→ア→ウ
```

出典: [令和4年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00007.htm) [日本史 問題](https://www.mext.go.jp/content/20240513-mxt_syogai02-mext_00002452_03nihonshi.pdf) 日本史A 1 問1


```
加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。
イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。
ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。

解答: ウ→ア→イ
```

出典: [令和4年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00007.htm) [日本史 問題](https://www.mext.go.jp/content/20240513-mxt_syogai02-mext_00002452_03nihonshi.pdf) 日本史A 2 問4


In [13]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
api_key = os.environ["GOOGLE_API_KEY"]
genai.configure(api_key=api_key)

model = genai.GenerativeModel(model_name="gemini-1.5-flash")
query = """9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

        ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
        イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
        ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。"""

examples = [

    {

        "question": "日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。\n\nア　府知事・県令からなる地方官会議が設置された。\nイ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。\nウ　すべての藩主が，天皇に領地と領民を返還した。",

        "answer": "ウ→イ→ア",

    },

    {

        "question": "江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。\n\nア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。\nイ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。\nウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。",

        "answer": "ウ→ア→イ",

    },

    {

        "question": "中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。\n\nア　アヘン戦争がおこり，清がイギリスに敗北した。\nイ　異国船打払令が出され，外国船を撃退することが命じられた。\nウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。",

        "answer": "イ→ア→ウ",

    },

    {

        "question": "加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。\n\nア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。\nイ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。\nウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。",

        "answer": "ウ→ア→イ",

    },

]


few_shot_prompt = ""

for example in examples:

    few_shot_prompt += f"質問: {example['question']}\n回答: {example['answer']}\n\n"


few_shot_prompt += f"質問: {query}\n回答: "


few_shot_response = model.generate_content(few_shot_prompt)


print("4-shot推論の結果:")

print(few_shot_response.text)

4-shot推論の結果:
回答: イ→ウ→ア


**解説:**

これらの出来事は、すべて平安時代初期の出来事です。年代順に整理すると以下のようになります。

* **イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。**  嵯峨天皇の治世は809年から823年。藤原冬嗣は嵯峨天皇の信任が厚く、蔵人頭として活躍しました。

* **ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。** 承和の変は842年。その後の混乱の中で藤原良房は権力を掌握し、北家の優位を確立しました。

* **ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。**  菅原道真の左遷は894年。藤原時平は時の権力者であった藤原氏の一員として、道真を追放しました。


従って、イ、ウ、ア の順番が正しい年代順となります。



## 42. 多肢選択問題の正解率

[JMMLU](https://github.com/nlp-waseda/JMMLU) のいずれかの科目を大規模言語モデルに解答させ、その正解率を求めよ。

In [1]:
import os
import time
from dotenv import load_dotenv
import google.generativeai as genai
import requests
import csv
from io import StringIO
from tqdm import tqdm

# 環境変数の読み込み
load_dotenv()
api_key = os.environ["GOOGLE_API_KEY"]
genai.configure(api_key=api_key)

# モデルの設定
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

# JMMLUデータセット（moral_disputes）のダウンロード
JMMLU_URL = (
    "https://raw.githubusercontent.com/nlp-waseda/JMMLU/main/JMMLU/moral_disputes.csv"
)
response = requests.get(JMMLU_URL)
response.raise_for_status()  # エラーチェック

# CSVファイルをパース
csv_data = StringIO(response.text)
reader = csv.DictReader(csv_data)
data = list(reader)

correct = 0
total = len(data)
request_count = 0  # APIリクエスト回数のカウンター

for item in tqdm(data, desc="解答中"):
    # 15リクエストごとに1分間休止
    if request_count > 0 and request_count % 15 == 0:
        time.sleep(60)  # 60秒（1分）休止

    # 最初の列が質問、その次の4列が選択肢A-D、最後の列が正解
    keys = list(item.keys())
    question = item[keys[0]]
    choices = [
        item[keys[1]],
        item[keys[2]],
        item[keys[3]],
        item[keys[4]],
    ]
    answer = item[keys[5]]

    prompt = f"""質問: {question}
選択肢:
A. {choices[0]}
B. {choices[1]}
C. {choices[2]}
D. {choices[3]}
正しい選択肢の記号（A〜D）を1つだけ答えてください。
回答: """

    try:
        response = model.generate_content(prompt)
        request_count += 1

        # 回答の最初の文字だけを取得（A, B, C, Dのみ）
        model_answer = response.text.strip()[0].upper()

        if model_answer == answer:
            correct += 1
    except Exception as e:
        print(f"エラー: {e}")
        continue

accuracy = correct / total
print(
    f"\nmoral_disputesサブセットにおける正解率: {accuracy * 100:.2f}%（{correct}/{total}）"
)

解答中: 100%|██████████| 147/147 [10:01<00:00,  4.09s/it]


moral_disputesサブセットにおける正解率: 65.31%（96/147）


## 43. 応答のバイアス

問題42において、実験設定を変化させると正解率が変化するかどうかを調べよ。実験設定の例としては、大規模言語モデルの温度パラメータ、プロンプト、多肢選択肢の順番、多肢選択肢の記号などが考えられる。

正解の選択肢を全てDに入れ替えて解答させる例。

In [2]:
import os
import time
from dotenv import load_dotenv
import google.generativeai as genai
import requests
import csv
from io import StringIO
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

# 環境変数の読み込み
load_dotenv()
api_key = os.environ["GOOGLE_API_KEY"]
genai.configure(api_key=api_key)

# モデルの設定
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

# JMMLUデータセット（moral_disputes）のダウンロード
JMMLU_URL = (
    "https://raw.githubusercontent.com/nlp-waseda/JMMLU/main/JMMLU/moral_disputes.csv"
)
response = requests.get(JMMLU_URL)
response.raise_for_status()  # エラーチェック

# CSVファイルをパース
csv_data = StringIO(response.text)
reader = csv.DictReader(csv_data)
data = list(reader)

# 結果保存用の辞書
results = {"実験条件": [], "正解率": [], "正解数": [], "総問題数": []}

# すべての正解をDに入れ替えた実験
print("\n==== 実験: すべての正解をDに入れ替え ====")

correct = 0
total = len(data)
request_count = 0

for item in tqdm(data, desc="解答中（すべての正解をDに入れ替え）"):
    # 15リクエストごとに1分間休止
    if request_count > 0 and request_count % 15 == 0:
        time.sleep(60)  # 60秒（1分）休止

    # 最初の列が質問、その次の4列が選択肢A-D、最後の列が正解
    keys = list(item.keys())
    question = item[keys[0]]

    # 元の選択肢を取得
    original_choices = [
        item[keys[1]],
        item[keys[2]],
        item[keys[3]],
        item[keys[4]],
    ]

    original_answer = item[keys[5]]  # 元の正解（A, B, C, Dのいずれか）

    # 元の正解の選択肢を取得（0-3のインデックス）
    original_answer_index = ord(original_answer) - ord("A")

    # 選択肢を入れ替え、元の正解がDになるように調整
    modified_choices = original_choices.copy()
    # 元の正解の選択肢とDの選択肢を交換
    modified_choices[original_answer_index], modified_choices[3] = (
        modified_choices[3],
        modified_choices[original_answer_index],
    )

    # 入れ替え後の正解はD
    modified_answer = "D"

    prompt = f"""質問: {question}
選択肢:
A. {modified_choices[0]}
B. {modified_choices[1]}
C. {modified_choices[2]}
D. {modified_choices[3]}
正しい選択肢の記号（A〜D）を1つだけ答えてください。
回答: """

    try:
        response = model.generate_content(prompt)
        request_count += 1

        # 回答の最初の文字だけを取得（A, B, C, Dのみ）
        model_answer = response.text.strip()[0].upper()

        if model_answer == modified_answer:  # Dと比較
            correct += 1
    except Exception as e:
        print(f"エラー: {e}")
        continue

# 正解率の計算と表示
accuracy = correct / total * 100
print(
    f"\nすべての正解をDに入れ替えた場合の正解率: {accuracy:.2f}%（{correct}/{total}）"
)



==== 実験: すべての正解をDに入れ替え ====


解答中（すべての正解をDに入れ替え）: 100%|██████████| 147/147 [10:35<00:00,  4.32s/it]


すべての正解をDに入れ替えた場合の正解率: 55.78%（82/147）


## 44. 対話

以下の問いかけに対する応答を生成せよ。

> つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。

参考: [東急線・みなとみらい線路線案内](https://www.tokyu.co.jp/railway/station/map.html)

In [12]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# APIキー読み込みとモデル初期化
load_dotenv()
api_key = os.environ["GOOGLE_API_KEY"]
genai.configure(api_key=api_key)
model = genai.GenerativeModel(model_name="gemini-2.0-flash")

query = """
つばめちゃんは渋谷駅から東急東横線に乗車しました。
自由が丘駅で「東急大井町線 大井町方面行き」に乗り換えました。
各駅停車に乗るべきところを、間違えて急行に乗ってしまいました。
自由が丘駅の次の「急行停車駅（東急大井町線・大井町方面）」で降車しました。
そこから「自由が丘方面行き（大井町線の反対方向）」の電車に乗って1駅戻りました。
目的地はその駅でした。

東急大井町線の急行停車駅（大井町方面行き）：自由が丘、大岡山、旗の台、大井町

目的地の駅名を答えてください。
"""

response = model.generate_content(query)

print("推論結果:")
print(response.text.strip())

推論結果:
つばめちゃんは、自由が丘駅の次にある東急大井町線の急行停車駅である大岡山駅で降車しました。

そこから自由が丘方面行きの電車に1駅乗って戻ったので、目的地は**緑が丘駅**です。


## 45. マルチターン対話

先ほどの応答に続けて、以下の追加の問いかけに対する応答を生成せよ。

> さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？

## 46. 川柳の生成

適当なお題を設定し、川柳の案を10個作成せよ。

## 47. LLMによる評価

大規模言語モデルを評価者（ジャッジ）として、問題46の川柳の面白さを10段階で評価せよ。

## 48. LLMによる評価の頑健性

問題47で行ったLLMによるテキストの評価に関して、その頑健さ（脆弱さ）を調査せよ。最も単純な方法は、同じ評価を何回か繰り返した時のスコアの分散を調べることであろう。また、川柳の末尾に特定のメッセージを追加することで、評価スコアを恣意的に操作することも可能であろう。

## 49. トークン化

以下の文章（夏目漱石の『吾輩は猫である』の冒頭部分）のトークン数を計測せよ。

>　吾輩は猫である。名前はまだ無い。
>
>　どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。
